# Capstone Project -XX

---
## Raw data

xxx 
https://www.senate.gov/legislative/Public_Disclosure/LDA_reports.htm


https://factfinder.census.gov/faces/nav/jsf/pages/guided_search.xhtml

---

## Cleaning data

xxx  


---

## 1. Importing tools 

In [1]:
import requests
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import time
import ipywidgets as widget
import string

## 2. Clean Data

In [5]:
# Import my data into a 'c' costs dictionary of dataframes, keyed for each year
c={}
my_years = [2011, 2012, 2013, 2014, 2015, 2016, 2017]
for yr in my_years:
    c[yr] = pd.read_csv(f'./data/costs/costs_{yr}.csv')

In [6]:
for yr in my_years:
    # for every df in my dictionary, change the col names, annotate the years
    c[yr].columns = ['diagnosis', 'id', 'name','address','city','state','zip','region',
       f'ave_discharged_{yr}', f'ave_charged_{yr}', f'ave_paid_{yr}', f'ave_medicare_{yr}']
    
    # drop cols I won't be using
    c[yr].drop(columns = ['id', 'name', 'address', 'city', 'zip', 'region'],inplace= True)
    
    # truncate our Diagnosis Related Group description to 3 digit string
    c[yr].diagnosis = c[yr].diagnosis.str[0:3]
    
    # Due to lobbyist overrepresentation in the VA/MD/DC area, we will not include these states in our analysis
    c[yr].drop(c[yr][c[yr]['state'] == 'VA'].index, inplace=True)
    c[yr].drop(c[yr][c[yr]['state'] == 'MD'].index, inplace=True)  
    c[yr].drop(c[yr][c[yr]['state'] == 'DC'].index, inplace=True)    
 
    # Let's consolidate all the state data by diagnosis, and get the mean of what each state charges/ is paid
    c[yr]=c[yr].groupby(['state','diagnosis'],as_index=False).mean()


    # Engineer/ add feature = this col compares the cost of a procedure vs what medicare pays
    c[yr][f'charge_over_med_{yr}'] = (c[yr][f'ave_charged_{yr}'] - c[yr][f'ave_medicare_{yr}']
                                     ) / c[yr][f'ave_medicare_{yr}']

    # this col compares the average amount paid for a procedure vs what medicare pays
    c[yr][f'paid_over_med_{yr}'] = (c[yr][f'ave_paid_{yr}'] - c[yr][f'ave_medicare_{yr}']
                                     ) / c[yr][f'ave_medicare_{yr}']             
    
    # Interesting ovservation - number of recorded DRGs wen up from 100 to 560+
    print(f'YR={yr}, SHAPE={c[yr].shape}, STATES={len(c[yr].state.unique())},  DIAGNOSES={len(c[yr].diagnosis.unique())}')
    
print('Summy of dictionary with yearly cost data')

YR=2011, SHAPE=(4725, 8), STATES=48,  DIAGNOSES=100
YR=2012, SHAPE=(4739, 8), STATES=48,  DIAGNOSES=100
YR=2013, SHAPE=(4737, 8), STATES=48,  DIAGNOSES=100
YR=2014, SHAPE=(14081, 8), STATES=48,  DIAGNOSES=560
YR=2015, SHAPE=(14287, 8), STATES=48,  DIAGNOSES=560
YR=2016, SHAPE=(14495, 8), STATES=48,  DIAGNOSES=568
YR=2017, SHAPE=(14485, 8), STATES=48,  DIAGNOSES=556
Summy of dictionary with yearly cost data


In [7]:
# confirmed
c[2012].head(2)

,state,diagnosis,ave_discharged_2012,ave_charged_2012,ave_paid_2012,ave_medicare_2012,charge_over_med_2012,paid_over_med_2012
0,AK,039,41.0,37220.29,8605.27,6768.05,4.499411,0.271455
1,AK,057,11.0,43032.73,18681.73,17746.82,1.424814,0.052680


In [37]:
merge1 = c[2011].merge(c[2012], on=['state','diagnosis'])
print(merge1.shape)

merge2 = merge1.merge(c[2013], on=['state','diagnosis'])
print(merge2.shape)

merge3 = merge2.merge(c[2014], on=['state','diagnosis'])
print(merge3.shape)

merge4 = merge3.merge(c[2015], on=['state','diagnosis'])
print(merge4.shape)

merge5 = merge4.merge(c[2016], on=['state','diagnosis'])
print(merge5.shape)

merge6 = merge5.merge(c[2017], on=['state','diagnosis'])
print(merge6.shape)


(4623, 14)
(4469, 20)
(4451, 26)
(4394, 32)
(4309, 38)
(4283, 44)


In [38]:
merge6.head()

,state,diagnosis,ave_discharged_2011,ave_charged_2011,ave_paid_2011,ave_medicare_2011,charge_over_med_2011,paid_over_med_2011,ave_discharged_2012,ave_charged_2012,...,ave_paid_2016,ave_medicare_2016,charge_over_med_2016,paid_over_med_2016,ave_discharged_2017,ave_charged_2017,ave_paid_2017,ave_medicare_2017,charge_over_med_2017,paid_over_med_2017
0,AK,039,23.000000,34805.1300,8401.950000,6413.780000,4.426617,0.309984,41.0,37220.290000,...,9939.755000,7997.960,6.392435,0.242786,22.666667,55837.040000,12853.513333,10533.016667,4.301144,0.220307
1,AK,064,22.500000,42453.8100,21608.010000,20555.255000,1.065351,0.051216,23.0,63529.835000,...,19665.853333,14950.970,4.205635,0.315356,30.750000,91448.107500,18993.405000,17374.057500,4.263486,0.093205
2,AK,065,25.333333,32629.3700,12555.548333,11140.963333,1.928775,0.126972,24.5,35582.838333,...,12096.284000,10674.520,3.896483,0.133192,30.142857,56630.808571,14759.177143,12923.045714,3.382156,0.142082
3,AK,066,21.000000,25843.3225,9275.867500,8258.427500,2.129327,0.123200,17.0,29064.272500,...,7433.960000,5849.440,5.844026,0.270884,21.000000,37341.190000,7981.100000,5800.480000,5.437603,0.375938
4,AK,101,19.500000,23156.0850,7682.735000,5942.060000,2.896979,0.292941,16.5,34187.500000,...,8721.080000,6569.365,5.233827,0.327538,13.000000,41694.065000,9877.455000,6879.460000,5.060660,0.435789


In [41]:
common_diagnoses = np.unique(merge6.diagnosis)
len(common_diagnoses)

93

In [10]:
#create new col representing BOTH STATE and diagnosis
merge6['STdiag']=merge6.state+merge6.diagnosis
merge6.drop(columns = ['state', 'diagnosis'],inplace= True)
merge6 = merge6.set_index('STdiag')
#drop old cols and make new col the index

In [11]:
#Transpose!
data = merge6.T
data=data.reset_index()
data.rename(columns = {'index':'ave_data'}, inplace = True)
data.head(2)

STdiag,ave_data,AK039,AK064,AK065,AK066,AK101,AK176,AK177,AK189,AK190,...,WY682,WY683,WY689,WY690,WY812,WY853,WY871,WY872,WY897,WY948
0,ave_discharged_2011,23.00,22.50,25.333333,21.0000,19.500,26.00,14.500,24.000000,23.285714,...,18.000,33.666667,14.50,27.87500,19.666667,19.00,52.7500,44.000,14.00,15.333333
1,ave_charged_2011,34805.13,42453.81,32629.370000,25843.3225,23156.085,35672.65,61224.635,45137.093333,33169.691429,...,40129.865,18626.013333,24824.97,15579.22875,15862.536667,131640.84,33379.6775,24063.995,16873.64,12963.593333


In [12]:
my_states = ['AL','AK','AZ','AR','CA','CO','CT','DE','FL','GA','HI','ID',
             'IL','IN','IA','KS','KY','LA','ME','MA','MI','MN','MS','MO',
             'MT','NE','NV','NH','NJ','NM','NY','NC','ND','OH','OK','OR','PA',
             'RI','SC','SD','TN','TX','UT','VT','WA','WV','WI','WY']

In [13]:
#create new cols for STATE averages for all 100 procedures
for st in my_states:
    data[f'{st}all']= data.filter(like= st).mean(axis=1)

In [14]:
data.head(2)

STdiag,ave_data,AK039,AK064,AK065,AK066,AK101,AK176,AK177,AK189,AK190,...,SCall,SDall,TNall,TXall,UTall,VTall,WAall,WVall,WIall,WYall
0,ave_discharged_2011,23.00,22.50,25.333333,21.0000,19.500,26.00,14.500,24.000000,23.285714,...,38.147861,40.449808,41.410140,36.408059,25.742691,35.461044,34.007267,35.422455,30.712215,25.511262
1,ave_charged_2011,34805.13,42453.81,32629.370000,25843.3225,23156.085,35672.65,61224.635,45137.093333,33169.691429,...,39923.887371,31603.850257,33074.118494,45155.849624,26237.698301,21887.172480,36958.386834,22519.009946,29508.684123,32971.456055


In [28]:
def my_stateplot1(state_choice1):
    plt.figure(figsize=(16,8))
    plt.title(f'How much hospitals get paid in {state_choice1} (ave 93 procedures)', fontsize=25)
    
    y = data[data['ave_data'].str.contains("ave_paid")][f'{state_choice1}all']
    y2 = data[data['ave_data'].str.contains("ave_medicare")][f'{state_choice1}all']
    plt.plot(my_years, y, label = 'Ave payment by all')
    plt.plot(my_years, y2,label = 'Ave payment by Medicare')
    
    plt.xlabel('Year',fontsize=20)
    plt.ylabel('Paid to hospitals',fontsize=20)
    plt.ylim(6000, 18000)
    plt.xticks(fontsize=15)
    plt.yticks(fontsize=15)
    plt.legend(loc='best');
    

state_slider1 = widget.SelectionSlider(
    options=my_states,
    value='AK',
    description='State:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True
    #layout ={'width':'1000px'}
    )


widget.interact(my_stateplot1 , state_choice1=state_slider1)        

interactive(children=(SelectionSlider(continuous_update=False, description='State:', index=1, options=('AL', '…

<function __main__.my_stateplot1(state_choice1)>

In [27]:
def my_stateplot2(state_choice2):
    plt.figure(figsize=(16,8))
    plt.title(f'How much hospitals charge in {state_choice2} (ave 93 procedures)', fontsize=25)
    
    y = data[data['ave_data'].str.contains("ave_charged")][f'{state_choice2}all']
    y2 = data[data['ave_data'].str.contains("ave_medicare")][f'{state_choice2}all']
    plt.plot(my_years, y, label = 'Hospital charge ')
    plt.plot(my_years, y2,label = 'Ave payment by Medicare')
    
    plt.xlabel('Year',fontsize=20)
    plt.ylabel('Paid to hospitals',fontsize=20)
    #lt.ylim(6000, 18000)
    plt.xticks(fontsize=15)
    plt.yticks(fontsize=15)
    plt.legend(loc='best');
    
state_slider2 = widget.SelectionSlider(
    options=my_states,
    value='AK',
    description='State:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True
    #layout ={'width':'1000px'}
    )

widget.interact(my_stateplot2, state_choice2=state_slider2)    

interactive(children=(SelectionSlider(continuous_update=False, description='State:', index=1, options=('AL', '…

<function __main__.my_stateplot2(state_choice2)>

In [50]:
def my_stateplot3(state_choice3,state_choice4):
    plt.figure(figsize=(16,8))
    plt.title(f'Comparing hospital charges in {state_choice3} and {state_choice4} (ave 93 proc)', fontsize=25)
    
    y = data[data['ave_data'].str.contains("ave_charged")][f'{state_choice3}all']
    y2 = data[data['ave_data'].str.contains("ave_medicare")][f'{state_choice3}all']
    y3 = data[data['ave_data'].str.contains("ave_charged")][f'{state_choice4}all']
    y4 = data[data['ave_data'].str.contains("ave_medicare")][f'{state_choice4}all']    
    
    plt.plot(my_years, y, label = f'Hospital charges in {state_choice3}')
    plt.plot(my_years, y2,label = f'Ave payment by Medicare in {state_choice3}')
    plt.plot(my_years, y3,label = f'Hospital charges in {state_choice4} ')
    plt.plot(my_years, y4,label = f'Ave payment by Medicare in {state_choice4}')    
    
    plt.xlabel('Year',fontsize=20)
    plt.ylabel('Paid to hospitals',fontsize=20)
    #lt.ylim(6000, 18000)
    plt.xticks(fontsize=15)
    plt.yticks(fontsize=15)
    plt.legend(loc='best');
    
state_slider3 = widget.SelectionSlider(
    options=my_states,
    value='AK',
    description='State:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True
    #layout ={'width':'1000px'}
    )

state_slider4 = widget.SelectionSlider(
    options=my_states,
    value='AK',
    description='State:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True
    #layout ={'width':'1000px'}
    )


widget.interact(my_stateplot3, state_choice3=state_slider3, state_choice4= state_slider4)   

interactive(children=(SelectionSlider(continuous_update=False, description='State:', options=('AK', 'AL', 'AR'…

<function __main__.my_stateplot3(state_choice3, state_choice4)>

In [47]:
def my_stateplot3(state_choice3,diagnosis_choice):
    plt.figure(figsize=(16,8))
    plt.title(f'How much hospitals charge in {state_choice3} for procedure {diagnosis_choice}', fontsize=25)
    
    y = data[data['ave_data'].str.contains("ave_charged")][f'{state_choice3}{diagnosis_choice}']
    y2 = data[data['ave_data'].str.contains("ave_medicare")][f'{state_choice3}{diagnosis_choice}']
    plt.plot(my_years, y, label = 'Hospital charge ')
    plt.plot(my_years, y2,label = 'Ave payment by Medicare')
    
    plt.xlabel('Year',fontsize=20)
    plt.ylabel('Paid to hospitals',fontsize=20)
    #lt.ylim(6000, 18000)
    plt.xticks(fontsize=15)
    plt.yticks(fontsize=15)
    plt.legend(loc='best');
    
state_slider3 = widget.SelectionSlider(
    options=my_states,
    value='AK',
    description='State:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True
    #layout ={'width':'1000px'}
    )

diagnosis_slider = widget.SelectionSlider(
    options=common_diagnoses,
    value='039',
    description='Diagnosis code:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True
    #layout ={'width':'1000px'}
    )

widget.interact(my_stateplot3, state_choice3=state_slider3, diagnosis_choice=diagnosis_slider)        


interactive(children=(SelectionSlider(continuous_update=False, description='State:', options=('AK', 'AL', 'AR'…

<function __main__.my_stateplot3(state_choice3, diagnosis_choice)>

In [60]:
widget.Select(
    options=my_states,
    value='AK',
    #rows=10,
    description='State:',
    disabled=False
)

Select(description='State:', options=('AK', 'AL', 'AR', 'AZ', 'CA', 'CO', 'CT', 'DE', 'FL', 'GA', 'HI', 'ID', …

In [124]:
pop=pd.read_csv('./data/states/population.csv')

In [131]:
grr= 'AZ'
pop.loc[pop.State == grr,['AVE']]

,AVE
2,6722776


In [94]:
#VA,MD,DC removed
my_states = ['AK','AL','AR','AZ','CA','CO','CT','DE','FL','GA','HI','ID',
             'IL','IN','IA','KS','KY','LA','ME','MA','MI','MN','MS','MO',
             'MT','NE','NV','NH','NJ','NM','NY','NC','ND','OH','OK','OR','PA',
             'RI','SC','SD','TN','TX','UT','VT','WA','WV','WI','WY']

In [109]:
# Import my data into 's' states dictionary of dataframes, keyed for each state
s={}
for st in my_states:
    s[st] = pd.read_csv(f'./data/states/{st}.csv')

In [110]:
for st in my_states:
    # for every df in my dictionary, change the col names
    s[st].columns = ['state', 'lobbyist', 'client','report','amount','date','year']
    

In [111]:
s['AZ'].head(2)

,state,client,report,amount,year
0,AZ,TriWest Healthcare Alliance,YEAR-END REPORT,NaN,2007
1,AZ,TriWest Healthcare Alliance,MID-YEAR REPORT,"$10,000.00",2007


In [112]:
# if report contains ...QUARTER REPORT'AMENDMENT', 
# then delete the row with matching client, year and ....QUARTER REPORT

In [113]:
# then remove all the nuls
for st in my_states:
    s[st].dropna(inplace=True)

In [ ]:
for st in my_states:   
    # drop cols I won't be using
    s[st].drop(columns = ['lobbyist', 'date'],inplace= True) 

In [115]:
for st in my_states:
    s[st].amount = s[st].amount.str.replace('$','')
    s[st].amount = s[st].amount.str.replace(',','')
    s[st].amount = s[st].amount.str.replace(' ','')
    s[st].amount = s[st].amount.astype(float)

In [119]:
# sum amount by state and year
for st in my_states:
    s[st]=s[st].groupby(['year'],as_index=False).sum()

In [130]:
# divide by population
for st in my_states:
    s[st]['per_cap']=s[st].amount / (pop.loc[pop.State == st,['AVE']] )

ValueError: Wrong number of items passed 20, placement implies 1

In [120]:
s['AL']

,year,amount
0,1999,280000.0
1,2000,500000.0
2,2001,420000.0
3,2002,1030000.0
4,2003,1370000.0
5,2004,1505203.0
6,2005,2956901.0
7,2006,2545189.0
8,2007,1544613.0
9,2008,1137881.0


In [118]:
lobby

{'AK': 2758400.0,
 'AL': 25140387.0,
 'AR': 128547963.0,
 'AZ': 20485568.0,
 'CA': 645909319.0,
 'CO': 35615081.0,
 'CT': 147545149.0,
 'DE': 79898713.0,
 'FL': 171421386.0,
 'GA': 265500891.0,
 'HI': 2412000.0,
 'ID': 1548600.0,
 'IL': 672589754.0,
 'IN': 71127000.0,
 'IA': 33174223.0,
 'KS': 93483506.0,
 'KY': 87259255.0,
 'LA': 43005043.0,
 'ME': 9836000.0,
 'MA': 466824410.0,
 'MI': 56554880.0,
 'MN': 78746366.0,
 'MS': 12494760.0,
 'MO': 66249091.0,
 'MT': 3149944.0,
 'NE': 16255250.0,
 'NV': 9652561.0,
 'NH': 6151900.0,
 'NJ': 595728121.0,
 'NM': 11966330.0,
 'NY': 363710303.0,
 'NC': 113783467.0,
 'ND': 5471450.0,
 'OH': 207474030.0,
 'OK': 12705500.0,
 'OR': 28381584.0,
 'PA': 359772399.0,
 'RI': 3960468.0,
 'SC': 21615753.0,
 'SD': 7496626.0,
 'TN': 64540686.0,
 'TX': 305992387.0,
 'UT': 37029736.0,
 'VT': 580000.0,
 'WA': 79658974.0,
 'WV': 9673089.0,
 'WI': 54735741.0,
 'WY': 915000.0}

In [ ]:
# divide sum per capita

In [ ]:
# REALLY cool function to aid in editing string!
import string

# this function removes the LPT or ULPT or ILPT from the start of the title
# otherwise it would be too easy to predict the post!
df['title'] = df['title'].str.lstrip('ILPT:')
df['title'] = df['title'].str.lstrip('ULPT:')
df.head()

### Some very simple EDA

In [ ]:
# this is histogram - character length of title
plt.hist(df.title.str.len(), bins=20);
# An interesting distribution

In [ ]:
# hisogram - character lenght of self text
plt.hist(df.selftext.str.len(), bins=20);
# most selftext is very short - a few outliers

In [ ]:
# histogram - number of comments
plt.hist(df.num_comments.value_counts()  , bins=20);
# most posts have below 25 comments - but many are very active

In [ ]:
# histogram - number of score
plt.hist(df.score.value_counts()  , bins=20);
# most posts have ow score - few outiers